In [1]:
import warnings
warnings.simplefilter('ignore')

import os
import re
import glob

import numpy as np
import pandas as pd
pd.set_option('max_rows', 200)
pd.set_option('max_colwidth', 400)

In [2]:
movie_dict = {
    '1171': '芳华',
    '1460': '风中有朵雨做的云',
    '2369': '日照重庆', 
    '2388': '超时空同居',
    '2721': '嫌疑人X的献身',
    '2930': '滚蛋吧肿瘤君',
    '2996': '集结号',
    '32504': '唐人街探案',
    '32505': '湄公河行动',
    '32795': '西虹市首富',
    '32798': '流浪地球',
    '32812': '一出好戏',
    '32845': '解救吾先生',
    '32899': '未知电影',
    '34121': '寻梦环游记',
    '34126': '那些年我们一起追的女孩',
    '34135': '霸王别姬',
    '34162': '狗十三',
    '34173': '红海行动',
    '34311': '哪吒之魔童降世',
    '34313': '卧虎藏龙',
    '34314': '寄生虫',
    '34527': '金陵十三钗',
    '34842': '绿皮书',
    '34899': '东京物语',
    '34911': '中国合伙人',
    '34913': '妖猫传',
    '34940': '天才枪手',
    '34945': '战狼',
    '34946': '熔炉',
    '34949': '紧急迫降',
    '34170': '中国机长',
    '34526': '悲伤逆流成河',
    '34893': '暴裂无声',
    '34896': '楚门的世界',
    '34898': '低俗小说',
    '34906': '追凶者也',
    '34907': '志明与春娇',
    '34909': '云水谣',
    '34914': '新喜剧之王',
    '1597': '泰囧',
    '2373': '亲爱的',
    '2719': '初恋这件小事',
    '2722': '无名之辈',
    '2731': '夏洛特烦恼',
    '3002': '疯狂的外星人',
    '3133': '扫毒',
    '34071': '烈日灼心',
    '34122': '我不是药神'
}

train_character_mappings = {
    '战狼': {
        'q2': '冷锋',
        'i3': '军官',
        'd3': '首长',
        'c3': '武吉',
        'j3': '警员',
        's1': '女监察员',
        't2': '老石',
        'u2': '队长',
        'k3': '教官',
        'g3': '人质',
        'l1': '狙击手',
        'c1': '龙小云',
        'w2': '敏登',
        'g1': '雇佣兵',
        'z2': '邵兵',
        'x2': '俞飞',
        'm3': '部下',
        'f3': '队员',
        'y2': '三八',
        'e3': '邵队',
        'm1': '二愣子',
        's2': '老猫',
        'w1': '男监察员',
        'l3': '察员',
        'r2': '小云'
    },
    '紧急迫降': {
        'r1': '马依俐',
        'x2': '舒骏',
        'n1': '季兰田',
        'b1': '李嘉棠', 
        'c1': '丘叶华',
        'g1': '周书记',
        'k1': '吴市长',
        'c3': '市长',
        'm1': '葛宏增',
        'j1': '何文滔',
        'w1': '章丙兆',
        'z1': '工作人员',
        'y1': '麦杰里',
        'u1': '赵雅旋',
        'w2': '刘远',
        'r3': '人员',
        's1': '指挥员',
        'p3': '机长',
        'a2': '马依俐',
        'v1': '乘务长',
        'a3': '空姐',
        'g3': '茜茜',
        's2': '魏朝成',
        'f1': '图拉其',
        'd3': '小飞',
        'h3': '孟凡',
        'h1': '副驾驶', 
        'z2': '驾驶', 
        'l3': '老头', 
        'i3': '少妇', 
        't3': '老太', 
        't2': '马依俐', 
        'j4': '吴为'
    },
    '嫌疑人X的献身': {
        'i2': '陈靖',
        'g2': '石泓',
        'f1': '陈晓欣',
        'h2': '唐川',
        'j2': '罗淼',
        'i1': '刑警',
        's2': '法医',
        'l2': '傅坚',
        'q2': '警察',
        'l1': '一个警察',
        'j1': '那警察',
        'r2': '店员',
        'm2': '坤哥',
        'k1': '那个警察',
        'k2': '晓欣',
        'n2': '队长'
    },
    '一出好戏': {
        'i2': '小兴',
        'h2': '马进',
        't2': '员工',
        'g1': '潘主任',
        'i1': '史教授',
        'd1': '总',
        'j1': '赵天龙',
        'k2': '姗姗',
        'c1': '王根基',
        'p2': '男人',
        'm2': '齐姐',
        'q2': '妇女',
        'o2': '天龙',
        'm1': '那男子',
        's2': '杨洪',
        'r2': '男子'
    },
    '解救吾先生':{
        'o2': '张华',
        'b1': '吾先生',
        'y2': '程总',
        'p2': '曹刚',
        'r2': '邢峰',
        'j1': '张国正',
        'w2': '刑警',
        'f3': '翻译',
        'p1': '李杰辉',
        's2': '小窦',
        't2': '小弟',
        'q2': '刑峰',
        'x2': '伟哥',
        'u2': '仓哥',
        'q1': '孔老大',
        'm1': '苏先生',
        'z2': '警察',
        'c3': '超子',
        'd3': '妻子',
        'e3': '陈晨',
        'v1': '那绑匪',
        'g3': '绑匪',
        'b3': '老大'
    },
    '绿皮书':{
        'v2': '利普',
        'j1': '罗斯古德',
        'u1': '卡迈恩',
        't3': '贡巴',
        'd1': '德洛丽丝',
        'z2': '强尼',
        'i3': '鲁迪',
        'm3': '弗兰',
        'v3': '托尼',
        'f3': '老板娘',
        'h3': '保利',
        'k1': '阿密特',
        'w2': '谢利',
        'r3': '鲍比',
        'p3': '奥基',
        'k3': '高管',
        'h1': '奥列格',
        'u3': '司机',
        'm1': '舞台经理',
        'c3': '一间背',
        'v1': '男人 1',
        'n1': '摩根安德',
        's1': '服务员',
        'y2': '警察 1',
        'x1': '多米尼克',
        'a3': '欣德',
        's3': '查理',
        'q3': '尼克',
        'g3': '德森'
    },
    '金陵十三钗': {
        'd1': '孟书娟',
        'g1': '女学生',
        'o2': '约翰',
        'f1': '日本兵',
        'i1': '李教官',
        'c1': '陈乔治',
        'c3': '女生甲',
        'w2': '红菱',
        'p2': '玉墨',
        'h1': '女人们',
        'b3': '怡春',
        'u1': '女人乙',
        'a3': '豆蔻',
        'z2': '女孩',
        'm1': '胖春',
        't1': '朝仓中尉',
        'j1': '孟先生',
        'x2': '大佐',
        'h3': '浦生',
        'e3': '泰瑞',
        'r1': '加藤中尉',
        't2': '学生',
        'r2': '书娟',
        'q2': '乔治',
        'y2': '春花'
    },
    '滚蛋吧肿瘤君': {
        'm2': '熊顿',
        'd3': '新娘',
        'u2': '男友',
        'x2': '老板',
        'q2': '艾米',
        'o2': '老郑',
        'z2': '美女',
        'p2': '小夏',
        't2': '熊妈',
        'w2': '熊爸',
        'n2': '医生',
        'v2': '护士',
        'r1': '小男孩',
        'n1': '爸爸',
        's2': '毛豆',
        'r2': '夏梦',
        'a3': '最后',
        's1': 'Waiter',
        'o1': '劈腿女',
        'b3': '男孩',
    },
    '超时空同居': {
        'l2': '小焦',
        'p1': '中介小马',
        'y2': '职员',
        'l1': '相亲男',
        'b3': '李总',
        'i1': '土豆大亨',
        'm2': '陆鸣',
        'c1': '赵俊以',
        'g1': '谷琪祥',
        'x2': '助理',
        'r2': '博士',
        't2': '店长',
        'p2': '小雅',
        'k1': '酒吧服务员',
        'a3': '保安',
        'd1': '陆石屹',
        's2': '大亨',
        'c3': '阿姨',
        'u2': '务员',
        'w2': '老公',
        'o2': '石屹',
        'n2': '俊以'
    },
    '那些年我们一起追的女孩': {
        'o2': '柯腾', 
        'q2': '阿和', 
        's2': '勃起', 
        't2': '老曹', 
        'v2': '教官', 
        'c1': '沈佳宜', 
        'r2': '弯弯', 
        'u2': '该边', 
        'y2': '赖导', 
        'i1': '老师',
        'c3': '邻居', 
        'v1': '柯景腾', 
        'q1': '室友义智', 
        'a3': '孝纶',
        'w2': '建伟',
        'z2': '新郎',
        'p2': '佳宜',
        'b3': '室友义智'
    },
    '湄公河行动': {
        'r2': '高刚',
        'g3': '部长',
        'i1': '郁局长',
        'g1': '宋比塔',
        'c1': '方新武',
        'j3': '瓦拉',
        't2': '糯卡',
        't3': '拿突',
        'u2': '大师',
        'h3': '队员',
        'b3': '二郎',
        'x2': '哪吒',
        's1': '快译通',
        'z2': '冰冰',
        'd3': '翁煞',
        'k3': '手下',
        'm1': '野牛队',
        'a3': '皮尔',
        'u1': '沙先生',
        'l3': '警察',
        'v2': '占蓬',
        'm3': '女儿',
        'e3': '桑吉',
        'f3': '依达',
        'n3': '船员',
        'w2': '比塔',
        'p3': '真人',
        'i3': '译通',
        's2': '新武',
        'y2': '局长'
    },
    '中国合伙人': {
        'a1': '成东青',
        'b1': '孟晓骏',
        'h2': '王阳',
        'j2': '良琴',
        'l2': '波诺',
        'q2': '女生',
        'o2': '记者',
        'i2': '苏梅',
        'k2': '男生',
        'p2': '张曦',
        'n2': '许文',
        'i1': '周知庸',
        'g2': '晓骏'
    },
    '妖猫传': {
        'x2': '春琴',
        'w2': '黑猫',
        'd1': '陈云樵',
        't2': '空海',
        'c1': '白乐天',
        'p3': '侍卫们',
        'r3': '看客',
        'r1': '中年男子',
        'm1': '金吾卫',
        'o3': '妓女',
        'i3': '玉莲',
        'k3': '丽香',
        'y1': '李少监',
        'n3': '牡丹',
        'u1': '老宫女',
        'l3': '白玲',
        'c3': '晁衡',
        'h3': '皇上',
        'z2': '白龙',
        'g3': '黄鹤',
        'a3': '丹龙',
        'j1': '高力士',
        'f3': '李白',
        'g1': '杨玉环',
        'l1': '李隆基',
        's3': '瓜翁',
        'v2': '云樵',
        'm3': '宫女',
        'u2': '乐天',
        'j3': '男子',
        'y2': '玉环'
    },
    '流浪地球': {
        'd1': '韩朵朵',
        'l2': '刘启',
        'f1': '韩子昂',
        'c1': '刘培强',
        'l1': '马卡洛',
        'h1': 'Tim',
        'i1': 'MOSS',
        'q1': '防护服',
        'r1': 'Moss',
        'k1': 'TIM',
        'o2': '王磊',
        't2': '周倩',
        'n1': '何连科',
        'w2': '锤子',
        'y2': '溜子',
        'p1': '外骨骼',
        'g1': '李一一',
        's1': '联合政府',
        'n2': '朵朵',
        'm2': '培强'
    },
    '红海行动': {
        'x1': '工作人员',
        'a3': '舰长',
        'c3': '陆琛',
        'r2': '杨锐',
        'v2': '佟莉',
        'x2': '李懂',
        'd3': '罗星',
        'u2': '徐宏',
        'y2': '石头',
        'j3': '政委',
        't2': '夏楠',
        'h3': '阿布',
        'f3': '威廉博士',
        'o3': '主编',
        'm3': '扎卡',
        'o1': '政府军',
        'w2': '顾顺',
        'q1': '何领事',
        'u1': '卢医生',
        'b3': '邓梅：',
        's2': '叛军',
        'l3': '司机',
        'i3': '武警',
        'z2': '庄羽',
        'n3': '人员',
        'g3': '领事'
    },
    '日照重庆': {
        'a1': '林权海',
        'p2': '老金',
        'h1': '李育英',
        'u2': '警察',
        's1': '中年男人',
        'a3': '淑平',
        'p1': '配货员',
        'x2': '保安',
        's2': '玲子',
        'y2': '表姐',
        't2': '方慧',
        'n2': '林波',
        'o2': '小雯',
        'b3': '医生',
        'm2': '强子',
        'w2': '男孩',
        'v2': '刘成',
        'q2': '竹青',
        'r2': '育英',
        'c3': '男人'
    },
    '哪吒之魔童降世': {
        'd1': '申公豹',
        'p1': '混元天尊',
        'l2': '李靖',
        'q2': '村民们',
        'h1': '李管家',
        'p2': '龙王',
        'i2': '哪吒',
        'v2': '兵士',
        'm1': '结界兽甲',
        'l1': '结界兽乙',
        'n2': '阿三',
        'n1': '猪头套',
        'r1': '太乙真人',
        'm2': '海妖',
        'r2': '小妹',
        'j2': '敖丙',
        'w2': '天尊',
        'u2': '小孩',
        'y2': '真人',
        'k2': '公豹'
    },
    '天才枪手': {
        'i2': '小琳',
        'e1': '格蕾丝',
        'f1': '老师',
        'k2': '小巴',
        'm1': '小林父亲',
        'j2': '班克',
        'n2': '校长',
        'm2': '小东',
        'o2': '考生',
        'g1': '考官'
    },
    '东京物语': {
        'm2': '周吉',
        't2': '京子',
        'n2': '富子',
        'y2': '主妇',
        'r2': '文子',
        'z2': '阿勇',
        's2': '阿实',
        'q2': '幸一',
        'o2': '志家',
        'p2': '纪子',
        'a3': '木代',
        'u2': '库造',
        'w2': '服部',
        'c3': '米子',
        'v2': '沼田',
        'b3': '加代',
        'x2': '敬三',
        's1': '老站务',
        'e3': '大夫',
        'd3': '务员'
    },
    '熔炉': {
        'b1': '姜仁浩',
        'g1': '母亲',
        'c1': '徐友真',
        'r1': '金教授',
        'm2': '校长',
        'i1': '行政室长',
        'e1': '朴宝贤',
        'k1': '张刑警',
        'q1': '工作人员',
        'f1': '黄律师',
        'u2': '被告',
        'p2': '法官',
        'n1': '母亲',
        'l1': '吴正植',
        'r2': '原告',
        'w2': '永勋',
        'j3': '慈爱',
        's2': '刑警',
        'a2': '金妍斗',
        'z1': '成宥利',
        'h3': '宥利',
        'i3': '妍斗',
        'k3': '民秀',
        'c2': '全民秀',
        'b2': '润慈爱',
        'o2': '律师',
        'y2': '人员',
        'q2': '室长',
        'v2': '翻译',
        'k2': '仁浩',
        'x2': '记者',
        'z2': '教授'
    },
    '卧虎藏龙': {
        'y2': '吴妈',
        'b1': '俞秀莲',
        'a1': '玉娇龙',
        'e1': '刘泰保',
        'g1': '蔡湘妹',
        'r2': '蔡九',
        'v2': '得禄',
        'm1': '铁小贝',
        'k1': '铁贝勒',
        'f1': '罗小虎',
        'n1': '李慕白',
        'm2': '秀莲',
        'x2': '慕白',
        'w2': '贝勒',
        'i1': '碧眼狐狸',
        'c3': '丫头',
        'n2': '慕白',
        'j1': '高师娘',
        'z2': '绣香',
        't2': '师娘',
        'p2': '小虎',
        'c1': '李慕白',
        'q2': '湘妹',
        's2': '狐狸',
        'l2': '娇龙',
        'r1': '米大彪',
        'q1': '鲁君雄'
    },
    '狗十三': {
        'f2': '李玩',
        'b1': '爸爸',
        'k1': '吴老师',
        'c1': '奶奶',
        'e1': '爷爷',
        'g2': '李堂',
        'i2': '高放',
        'l2': '摊贩',
        'm2': '教练',
        'j2': '昭昭',
        'k2': '张哥',
        'h2': '后妈'
    },
    '集结号': {
        'a1': '谷子地',
        'g1': '指导员',
        'd1': '焦大棚',
        'h1': '吕宽沟',
        'i1': '姜茂财',
        'm1': '焦大棚',
        'f1': '王金存',
        'q1': '姜茂才',
        'l1': '罗广田',
        'j1': '曹排长',
        'c1': '赵二斗',
        'u2': '排长',
        'w2': '广田',
        's2': '宽沟',
        't2': '茂财',
        's1': '干部甲',
        'c3': '教员',
        'k1': '小梁子',
        'v2': '梁子',
        'y2': '龚良国',
        'q2': '金存',
        'b3': '茂财',
        'r1': '刘干事', 
        'e1': '孙桂琴',
        'o1': '李参谋',
        't1': '朱矿长',
        'n2': '二斗',
        'p1': '秦主任',
        'a3': '主任',
    },
    '风中有朵雨做的云': {
        'u1': '青年甲',
        'x2': '男性',
        'a1': '杨家栋',
        'd1': '唐奕杰',
        'u2': '警察',
        'w2': '记者',
        'v2': '秘书：',
        'q1': '女居户',
        'p2': '林慧',
        's2': '小诺',
        'e1': '姜紫成',
        'i3': '空境',
        't2': '阿云',
        'm1': '电视音',
        'n1': '工作人员',
        'c3': '顾客',
        'f3': '男士',
        'y2': '侦探',
        'o1': '播音员',
        't1': '快递员',
        'r1': '胡小姐',
        'a3': '人员',
        'y1': '唐奕杰',
        'e3': '小姐',
        'q2': '奕杰',
        'r2': '紫成',
        'o2': '家栋'
    },
    '唐人街探案': {
        'l2': '秦风',
        'k2': '唐仁',
        'x2': '王婆',
        'c1': '黄兰登',
        's2': '警员',
        'n2': '坤泰',
        'n1': '警局大人',
        'o2': '阿香',
        't2': '托尼',
        'm1': '越南仔',
        'q2': '北哥',
        'r2': '金刚',
        'w2': '店员',
        'p2': '思诺',
        'y2': '大哥',
        'p1': '闫先生',
        'i1': '李',
        'm2': '兰登',
        'v2': '大人'
    },
    '寻梦环游记': {
        'l2': '骷髅',
        'i2': '米高',
        'g1': '奶奶',
        'f1': '伊美黛',
        'n2': '苛刻',
        'd1': '德拉库斯',
        'o1': '父亲',
        'i1': '工作人员',
        'n1': '罗西塔',
        'k1': '胡里欧',
        'r2': '主任',
        'p2': '保安',
        'j2': '海特',
        'm1': '猪皮哥',
        'p1': '佛里'
    },
    '寄生虫': {
        'a1': '金基宇',
        'e1': '金基婷',
        'm2': '忠淑',
        'b1': '金基泽',
        'u2': '老板',
        'v2': '醉汉',
        't2': '敏赫',
        'o2': '雯光',
        'c1': '朴太太',
        'j1': '朴多慧',
        'k1': '朴多颂',
        'h1': '朴社长',
        'o1': '尹司机',
        'i1': '吴勤世',
        'x2': '警察',
        'y2': '医生',
        'n2': '基婷',
        'j2': '基宇',
        'r2': '多慧',
        'l2': '太太',
        's2': '多颂',
        'w2': '司机'
    },
    '霸王别姬': {
        'u1': '大花脸',
        'i3': '艳红',
        's2': '班主',
        'g1': '小石头',
        'z2': '红卫兵',
        'e1': '小豆子',
        'k1': '小癞子',
        'y2': '师父',
        'u2': '那坤',
        'n1': '张公公',
        'a1': '段小楼',
        'b1': '程蝶衣',
        'p1': '甲',
        'j1': '袁世卿',
        'r1': '老鸨子',
        'h3': '彩凤',
        'q2': '菊仙',
        'b3': '警察长',
        'v2': '小四',
        'f3': '法官',
        's1': '红卫兵',
        'q1': '逼问者',
        'r2': '豆子',
        'x2': '癞子',
        't2': '石头',
        'a3': '公公',
        'p2': '蝶衣',
        'o2': '小楼',
        'w2': '世卿',
        'e3': '卫兵'
    },
    '芳华': {
        'o2': '刘峰',
        'c1': '何小萍',
        'b1': '萧穗子',
        'h3': '队长',
        'f1': '林丁丁',
        'x2': '卓玛',
        'd1': '郝淑雯',
        'y2': '男兵',
        'u2': '陈灿',
        'q1': '摄像师',
        'j1': '宁政委',
        'b3': '朱克',
        'e3': '何小萍',
        'm1': '那士兵',
        't1': '石林峰',
        'c3': '医生',
        's1': '何小萍',
        'p2': '穗子',
        'q2': '小萍',
        's2': '丁丁',
        'i3': '首长',
        'w2': '政委',
        'z2': '士兵',
        'f3': '警察'
    },
    '西虹市首富': {
        'e1': '金先生',
        'a1': '王多鱼',
        'o2': '教练',
        'm2': '庄强',
        'v2': '司机',
        'm1': '那男子',
        'l2': '夏竹',
        'j1': '殷先生',
        'h1': '赖先生',
        'q2': '保安',
        'i1': '服务员',
        'q1': '经理',
        'f1': '柳建南',
        'r1': '巴菲特',
        'r2': '高然',
        'n1': '解说员',
        'o1': '工作人员',
        'k2': '多鱼', 
        's2': '男子', 
        'n2': '建南', 
        'y2': '真人', 
        'u2': '人员'
    }
}

test_character_mappings = {
    '志明与春娇': {
        's2': '保安',
        'm2': '警察',
        'a1': '余春娇',
        'o2': '小红',
        'k2': '公公',
        'p1': '广告客户',
        'j1': '服务员',
        'b1': '张志明',
        'p2': '惠英',
        'g1': '收银员',
        'l2': '家豪',
        'u2': '裴娜',
        'k1': '那朋友',
        'n1': '警官',
        'm1': '同事乙',
        'r2': '朋友',
        'j2': '志明',
        'i2': '春娇'
    },
    '楚门的世界': {
        'a1': '杜鲁门',
        'i1': '父亲',
        'p2': '马隆',
        'n1': '工作人员',
        't2': '女人',
        'd1': '梅丽尔',
        'f1': '母亲',
        'r2': '劳伦',
        'c1': '克里斯托弗',
        't1': '斯潘瑟',
        'u2': '男人',
        'r1': '主持人',
        'x2': '护士',
        'z2': '警察',
        'v2': '柯克',
        'j1': '西尔维娅',
        'q2': '西蒙',
        'y2': '克罗',
        'b3': '莫斯',
        'w2': '人员'
    },
    '无名之辈': {
        'f1': '肇红霞',
        't2': '警察',
        'b1': '胡广生',
        'c1': '李海根',
        'a1': '马先勇',
        'b3': '刘五',
        'j1': '王顺才',
        'd1': '马嘉旗',
        'a3': '仁队',
        'v2': '高明',
        's1': '老师',
        'h1': '马依依',
        'z2': '高翔',
        'c3': '保安',
        'd3': '老黄',
        'x2': '任队',
        'g3': '这时',
        'l1': '陈波波',
        'f3': '桥下',
        'e3': '人群', 
        'h3': '妇女', 
        'u2': '依依', 
        'r2': '嘉旗'
    },
    '悲伤逆流成河': {
        'i2': '易遥',
        'j2': '齐铭',
        'h1': '女孩儿',
        'e1': '唐小米',
        'f1': '顾森湘',
        'r2': '医生',
        'c1': '顾森西',
        'j1': '老师',
        'o2': '女生',
        'q2': '男生',
        's2': '校长',
        'p2': '大叔',
        't2': '警察',
        'n2': '孩儿', 
        'l2': '小米', 
        'm2': '森湘', 
        'k2': '森西'
    },
    '烈日灼心': {
        'a1': '辛小丰',
        'c1': '杨自道',
        'g1': '陈比觉',
        't2': '矮个',
        'r2': '警察',
        'b1': '伊谷春',
        'm2': '尾巴',
        'u2': '老头',
        'q2': '嫌犯',
        'e1': '伊谷夏',
        'o2': '师父',
        's2': '协警',
        'w2': '司机',
        'o1': '服务员',
        'p2': '何松',
        'v2': '务员', 
        'i2': '小丰', 
        'j2': '谷春', 
        'n2': '比觉', 
        'k2': '自道'
    },
    '低俗小说': {
        'k3': '姑娘',
        'q1': '小伙子',
        'r3': '顾客',
        'k1': '南瓜仔',
        'a1': '文森特',
        'b1': '朱尔斯',
        'g1': '马塞勒斯',
        'm1': '布雷特',
        'a2': '第四者',
        'w2': '巴奇',
        'u1': '英国人戴维',
        'g3': '乔迪',
        'a3': '兰斯',
        'x2': '米亚',
        't3': '巴蒂',
        'o1': '艾思玛瑞',
        'f1': '法比恩妮',
        'l1': '梅纳德',
        'i3': '齐德',
        'o3': '马文',
        'b3': '吉米',
        'j1': '沃尔夫',
        's1': '温斯顿',
        'w1': '拉奎尔',
        't1': '小兔子',
        'z1': '经理',
        'x1': '约兰达',
        'n3': '戴维',
        'h3': '瑞达',
        'm3': '兔子'
    },
    '疯狂的外星人': {
        'o2': '扎克',
        't2': '老板',
        'c1': '外星人',
        'k2': '耿浩',
        'x2': '大哥',
        'l2': '大飞',
        'o1': '经理',
        'r1': '按摩师',
        'n2': '约翰',
        'q2': '黑人保镖',
        'u2': '专家',
        'p2': '保镖',
        'j1': '工作',
        'w2': '主任',
        'v2': '刘总',
        'y2': '老廖',
        'r2': '警察',
        'm2': '外星人',
        'b3': '真人'
    },
    '初恋这件小事': {
        'z2': '阿亮',
        'u1': '女生们',
        'c2': '某女生',
        'b3': '阿霞',
        'y2': '小水',
        'k1': '妈妈',
        'g3': '小粉',
        'h1': '好友甲',
        'd3': '阿宁',
        'j3': '男生',
        'a3': '小茵',
        'f3': '小菲',
        'r3': '象叔',
        's3': '阿顶',
        'w3': '阿猫',
        'u3': '主任',
        't1': '三女生',
        'h3': '阿鹏',
        'o3': '小欧',
        'o1': '好友们',
        'i3': '小彬',
        'p1': '理事长',
        'x1': '某同学',
        'c3': '阿拓',
        'z3': '小梦',
        'v3': '阿船',
        'r1': '主持人',
        'x3': '女生', 
        'k3': '友们', 
        't3': '同学', 
        'a2': '导', 
        'f2': '王', 
        'q3': '生们'
    },
    '云水谣': {
        'f1': '孟晓芮',
        'b1': '王碧云',
        'a1': '陈秋水',
        'i1': '王太太',
        'k1': '王雨萌',
        'e1': '孟子路',
        'g1': '王庭武',
        'h1': '徐凤娘',
        't2': '莉莉',
        'd1': '王金娣',
        'u2': '司机',
        'w2': '女佣',
        'o1': '老馆员',
        's2': '卓玛',
        'j1': '陈昆仑',
        'm2': '晓芮', 
        'r2': '雨萌', 
        'i2': '秋水', 
        'p2': '太太', 
        'n2': '庭武', 
        'l2': '子路', 
        'j2': '碧云', 
        'k2': '金娣'
    },
    '泰囧': {
        'h2': '徐朗',
        'k2': '安安',
        'j2': '高博',
        'l2': '莉莉',
        't2': '小迪',
        'i2': '王宝',
        'p2': '空姐',
        'u2': '导游',
        'q2': '司机',
        'h1': '前台小姐',
        'r2': '商贩',
        'g1': '小曼',
        'm1': '修理工',
        'i1': '范冰冰',
        'm2': '外音',
        'y2': '周扬',
        'o2': '冰冰'
    },
    '追凶者也': {
        'x2': '猫哥',
        'i3': '司机',
        'f3': '法医',
        'u2': '警察',
        'j1': '何清勇',
        'r2': '老二',
        'e3': '宋涛',
        'g1': '钱贵兴',
        'm1': '马晓兰',
        'h3': '大爷',
        'd3': '老板',
        'g3': '小陈',
        'b3': '村民',
        'a3': '村长',
        'c1': '王友全',
        'j3': '勇哥',
        'v1': '老太太',
        'o3': '毛豆',
        'e1': '杨淑华',
        's2': '董小凤',
        'y2': '小萍',
        'k3': '凤哥',
        'm3': '舞女',
        'n3': '特警'
    },
    '中国机长': {
        'b1': '刘长健',
        'i3': '女儿',
        'f1': '工作人员',
        'v2': '毕男',
        'w2': '梁栋',
        'e1': '徐奕辰',
        'r3': '主任',
        'v1': '气象员',
        'o1': '乘务员',
        'f3': '杨慧',
        'z2': '黄佳',
        'w1': '张秋月',
        'i1': '周雅雯',
        'c3': '妻子',
        'l1': '乘客们',
        'u3': '阿姨',
        'g3': '妇女',
        'n3': '丈夫',
        'r1': '藏族男',
        'q1': '背头男',
        's1': '白胡子老头',
        'm3': '壮汉',
        'y2': '人员',
        'b3': '雅雯'
    },
    '暴裂无声': {
        'p2': '丁海',
        'r2': '村长',
        'o2': '翠霞',
        'w2': '校长',
        'b1': '昌万年',
        'c1': '徐文杰',
        'b3': '前台',
        'q2': '媛媛',
        's2': '李总',
        'n2': '金哥',
        'u2': '警察',
        'i1': '母亲',
        'v2': '大厨',
        'p1': '检察员',
        't2': '栓子',
        'x2': '司机',
        'r1': '那手下',
        'z1': '张保民', 
        'i3': '保民', 
        'z2': '妻子',
        'a3': '手下', 
        'm2': '文杰'
    },
    '新喜剧之王': {
        'l3': '大爷',
        'n3': '司机',
        'h3': '演员',
        'b3': '群演',
        'v2': '如梦',
        'z1': '男人2',
        'c3': '招工',
        'y2': '李洋',
        'x2': '查理',
        'a3': '小米',
        'p3': '顾客',
        'f3': '师傅',
        'g1': '工作人员',
        't3': '老板',
        'w2': '马可',
        'm3': '大妈',
        'e3': '盒饭',
        'r3': '王子',
        'j3': '苏导',
        'o1': '陈导演',
        'v1': '老师',
        'o3': '保安',
        's1': '女粉丝',
        'z2': '人员', 
        'g3': '导演', 
        's3': '场务', 
        'd3': '助理', 
        'i3': '马可',
        'k3': '粉丝'
    },
    '夏洛特烦恼': {
        'm2': '夏洛',
        'p2': '大春',
        'o2': '邱雅',
        'w2': '婉婉',
        't2': '孟特',
        'q2': '袁华',
        'n2': '冬梅',
        'f1': '王老师',
        'r2': '张扬',
        'y2': '新郎',
        'a3': '贝贝',
        'v2': '校长',
        'c3': '王姨',
        'b3': '小弟',
        'u2': '陈凯',
        'z2': '司机',
        'd3': '那姐',
        'x2': '护士'        
    },
    '扫毒': {
        'a1': '苏建秋',
        'r2': '警察',
        'q2': '黑柴',
        'b1': '马昊天',
        'c1': '张子伟',
        'v2': '小弟们',
        'd3': '卡泥',
        't1': '黄警官',
        'g1': '八面佛',
        't2': '波比',
        'b3': '苏建秋',
        'r1': '指挥官',
        'w2': '益哥',
        'y2': '沙立',
        'x2': '缅娜',
        'u2': '段坤',
        'a3': '长官', 
        'z2': '妻子',
        'p2': '子伟',
        'e3': '媛媛'
    },
    '我不是药神': {
        'e3': '老板',
        'u2': '程勇',
        'm1': '程小澍',
        'g3': '律师',
        'j3': '前妻',
        'p3': '警员',
        'w2': '曹斌',
        'b1': '吕受益',
        'i3': '医生',
        'z1': '王主任',
        'n3': '保安',
        't1': '詹姆斯',
        'o3': '船员',
        'y2': '思慧',
        'l3': '群主',
        'i1': '刘牧师',
        'x2': '彭浩',
        'k3': '病友',
        'r3': '黄队',
        'd3': '局长',
        'o1': '赵立忠',
        'w1': '酒吧经理',
        'g1': '张长林',
        'b2': '张院士',
        'a3': '警察',
        'f3': '小澍',
        'c3': '妻子'
    },
    '亲爱的': {
        'w1': '快递员',
        'm3': '小珍',
        'a1': '田文军',
        'c3': '男人',
        'g3': '女人',
        'd1': '鲁晓娟',
        'u3': '小宇',
        'o1': '母亲',
        'd3': '警察',
        'n3': '医生',
        'i3': '房东',
        'f1': '韩德忠',
        'z1': '田文军',
        'h3': '丈夫',
        'k3': '樊芸',
        'c1': '李红琴',
        'o3': '村民',
        'g1': '刘向东',
        'b3': '吉芳',
        'e3': '院长',
        'y2': '高夏',
        'r3': '前台',
        't3': '主任',
        'l1': '唐青山',
        's1': '工作人员',
        'j3': '法官',
        'x1': '女律师',
        'v3': '田鹏',
        'a4': '鹏鹏',
        'x2': '晓娟', 
        'o2': '韩德忠', 
        'i4': '吉刚', 
        'q2': '杨明富', 
        'p2': '杨吉刚', 
        'q3': '律师', 
        'l3': '人员', 
        'f3': '青山', 
        's3': '鼓掌'
    }
}

In [3]:
train = pd.read_csv('raw_data/train_dataset_v2.tsv', sep='\t')
tmp = pd.DataFrame(train['id'].str.split('_', expand=True))
tmp.columns = ['movie', 'scene', 'A', 'movie_id']
for col in ['movie', 'scene', 'movie_id']:
    train[col] = tmp[col].values
train['movie_id'] = train['movie_id'].astype(int)
train['movie'] = train['movie'].map(movie_dict)
train = train.sort_values(by=['movie', 'movie_id'])
train['character'].fillna('无角色', inplace=True)
train['emotions'].fillna('', inplace=True)

# 异常文本清理下
train['content'] = train['content'].apply(lambda x: x.replace('waitc3', 'waiter'))
train['content'] = train['content'].apply(lambda x: x.replace('undc3stand', 'understand'))
train['content'] = train['content'].apply(lambda x: x.replace('s0', 'OS'))
train['content'] = train['content'].apply(lambda x: x.replace('wi11', 'will'))
train['content'] = train['content'].apply(lambda x: x.replace('女检l3', '女监l3'))
train['content'] = train['content'].apply(lambda x: x.replace('老r2：老猫', '龙r2：老猫'))
train['content'] = train['content'].apply(lambda x: x.replace('M国', '美国'))
train['content'] = train['content'].apply(lambda x: x.replace('m国', '美国'))

train.head(10)

,id,content,character,emotions,movie,scene,movie_id
14632,32812_0001_A_1,一颗卫星的摄像头对准了一望无际的太空深处。,无角色,,一出好戏,0001,1
14633,32812_0001_A_2,一颗巨大的陨石穿过碎石阵朝黑暗深处快速降落。,无角色,,一出好戏,0001,2
14634,32812_0001_A_3,一个美国主持人报道着新闻的画外音：最新消息，美国宇航局之前观测的陨石，受到撞击后改变运行轨迹，正朝地球高速飞来，专家认为，陨石撞击地球的几率非常高。,无角色,,一出好戏,0001,3
14635,32812_0001_A_4,一个B国主持人的画外音：政府已经做好了万全准备，一定尽最大努力确保大家的安全。,无角色,,一出好戏,0001,4
14636,32812_0001_A_5,更多国家主持人不同语言的声音交叉在一切，都在报道陨石事件。,无角色,,一出好戏,0001,5
14637,32812_0001_A_6,一个中国主持人的画外音：接下来我们请著名专家i1来分析一下这次的陨石事件。,无角色,,一出好戏,0001,6
14638,32812_0001_A_7,i1画外音：如果这颗陨石降落到海洋，它掀起的海啸高度可能达到一百米左右，不过呢，这种概率是很低的，我就听网友调侃说，这是外太空送来的一份外卖。,i1,"0,0,0,0,0,0",一出好戏,0001,7
14640,32812_0002_A_8,i2在修理着一辆黑色面包车。,i2,"0,0,0,0,0,0",一出好戏,0002,8
14639,32812_0002_A_9,h2坐在车里面听着和陨石事件有关的广播：还整出个世界末日来，光脚的不怕穿鞋的，我跟你说那陨石要是真掉下来，损失最小的就是咱们这种没钱的，让你买彩票买了吗？,h2,"0,0,0,0,0,0",一出好戏,0002,9
14642,32812_0002_A_10,i2：买了，早买了，放k2旁白了，还是老号码。,i2,"0,0,0,0,0,0",一出好戏,0002,10


In [4]:
def replace_character(character, movie, datatype='train'):
    if datatype == 'train':
        mappings = train_character_mappings
    else:
        mappings = test_character_mappings
    if movie not in mappings:
        return character
    if character in mappings[movie]:
        return mappings[movie][character]
    else:
        return character

In [5]:
train['character_new'] = train.apply(lambda row: replace_character(row['character'], row['movie']), axis=1)
train.head(10)

,id,content,character,emotions,movie,scene,movie_id,character_new
14632,32812_0001_A_1,一颗卫星的摄像头对准了一望无际的太空深处。,无角色,,一出好戏,0001,1,无角色
14633,32812_0001_A_2,一颗巨大的陨石穿过碎石阵朝黑暗深处快速降落。,无角色,,一出好戏,0001,2,无角色
14634,32812_0001_A_3,一个美国主持人报道着新闻的画外音：最新消息，美国宇航局之前观测的陨石，受到撞击后改变运行轨迹，正朝地球高速飞来，专家认为，陨石撞击地球的几率非常高。,无角色,,一出好戏,0001,3,无角色
14635,32812_0001_A_4,一个B国主持人的画外音：政府已经做好了万全准备，一定尽最大努力确保大家的安全。,无角色,,一出好戏,0001,4,无角色
14636,32812_0001_A_5,更多国家主持人不同语言的声音交叉在一切，都在报道陨石事件。,无角色,,一出好戏,0001,5,无角色
14637,32812_0001_A_6,一个中国主持人的画外音：接下来我们请著名专家i1来分析一下这次的陨石事件。,无角色,,一出好戏,0001,6,无角色
14638,32812_0001_A_7,i1画外音：如果这颗陨石降落到海洋，它掀起的海啸高度可能达到一百米左右，不过呢，这种概率是很低的，我就听网友调侃说，这是外太空送来的一份外卖。,i1,"0,0,0,0,0,0",一出好戏,0001,7,史教授
14640,32812_0002_A_8,i2在修理着一辆黑色面包车。,i2,"0,0,0,0,0,0",一出好戏,0002,8,小兴
14639,32812_0002_A_9,h2坐在车里面听着和陨石事件有关的广播：还整出个世界末日来，光脚的不怕穿鞋的，我跟你说那陨石要是真掉下来，损失最小的就是咱们这种没钱的，让你买彩票买了吗？,h2,"0,0,0,0,0,0",一出好戏,0002,9,马进
14642,32812_0002_A_10,i2：买了，早买了，放k2旁白了，还是老号码。,i2,"0,0,0,0,0,0",一出好戏,0002,10,小兴


In [6]:
def replace_content(content, movie, datatype='train'):
    if datatype == 'train':
        mappings = train_character_mappings
    else:
        mappings = test_character_mappings
    if movie not in mappings:
        return content
    text = content
    for ch in mappings[movie]:
        text = text.replace(ch, mappings[movie][ch])
    return text

In [7]:
train['content_new'] = train.apply(lambda row: replace_content(row['content'], row['movie']), axis=1)
train.head(10)

,id,content,character,emotions,movie,scene,movie_id,character_new,content_new
14632,32812_0001_A_1,一颗卫星的摄像头对准了一望无际的太空深处。,无角色,,一出好戏,0001,1,无角色,一颗卫星的摄像头对准了一望无际的太空深处。
14633,32812_0001_A_2,一颗巨大的陨石穿过碎石阵朝黑暗深处快速降落。,无角色,,一出好戏,0001,2,无角色,一颗巨大的陨石穿过碎石阵朝黑暗深处快速降落。
14634,32812_0001_A_3,一个美国主持人报道着新闻的画外音：最新消息，美国宇航局之前观测的陨石，受到撞击后改变运行轨迹，正朝地球高速飞来，专家认为，陨石撞击地球的几率非常高。,无角色,,一出好戏,0001,3,无角色,一个美国主持人报道着新闻的画外音：最新消息，美国宇航局之前观测的陨石，受到撞击后改变运行轨迹，正朝地球高速飞来，专家认为，陨石撞击地球的几率非常高。
14635,32812_0001_A_4,一个B国主持人的画外音：政府已经做好了万全准备，一定尽最大努力确保大家的安全。,无角色,,一出好戏,0001,4,无角色,一个B国主持人的画外音：政府已经做好了万全准备，一定尽最大努力确保大家的安全。
14636,32812_0001_A_5,更多国家主持人不同语言的声音交叉在一切，都在报道陨石事件。,无角色,,一出好戏,0001,5,无角色,更多国家主持人不同语言的声音交叉在一切，都在报道陨石事件。
14637,32812_0001_A_6,一个中国主持人的画外音：接下来我们请著名专家i1来分析一下这次的陨石事件。,无角色,,一出好戏,0001,6,无角色,一个中国主持人的画外音：接下来我们请著名专家史教授来分析一下这次的陨石事件。
14638,32812_0001_A_7,i1画外音：如果这颗陨石降落到海洋，它掀起的海啸高度可能达到一百米左右，不过呢，这种概率是很低的，我就听网友调侃说，这是外太空送来的一份外卖。,i1,"0,0,0,0,0,0",一出好戏,0001,7,史教授,史教授画外音：如果这颗陨石降落到海洋，它掀起的海啸高度可能达到一百米左右，不过呢，这种概率是很低的，我就听网友调侃说，这是外太空送来的一份外卖。
14640,32812_0002_A_8,i2在修理着一辆黑色面包车。,i2,"0,0,0,0,0,0",一出好戏,0002,8,小兴,小兴在修理着一辆黑色面包车。
14639,32812_0002_A_9,h2坐在车里面听着和陨石事件有关的广播：还整出个世界末日来，光脚的不怕穿鞋的，我跟你说那陨石要是真掉下来，损失最小的就是咱们这种没钱的，让你买彩票买了吗？,h2,"0,0,0,0,0,0",一出好戏,0002,9,马进,马进坐在车里面听着和陨石事件有关的广播：还整出个世界末日来，光脚的不怕穿鞋的，我跟你说那陨石要是真掉下来，损失最小的就是咱们这种没钱的，让你买彩票买了吗？
14642,32812_0002_A_10,i2：买了，早买了，放k2旁白了，还是老号码。,i2,"0,0,0,0,0,0",一出好戏,0002,10,小兴,小兴：买了，早买了，放姗姗旁白了，还是老号码。


In [8]:
def check_(x):
    return re.findall(r'[a-z][0-9]', x)

train['not_changes'] = train['content_new'].apply(lambda x: check_(x))
train['not_changes'] = train['not_changes'].apply(lambda x: list(set(x)))
train['not_changes_len'] = train['not_changes'].apply(len)
train['not_changes'] = train['not_changes'].apply(lambda x: ','.join(x))

In [9]:
tmp = train[train['not_changes_len'] > 0].copy()
tmp.groupby('movie')['not_changes'].unique()

movie
未知电影    [y2, p1, a1, b1, a1,b1, u2, f1, u2,n2, u2,a1,b1, u2,w2,a1, w2,b1, u2,a1, w2,a1, u2,a1,j2, w2, u2,a1,n2, p2, a1,b1,n2,p2, n2,p2, a1,b1,n2, w2,n2,b1, b1,p2, a1,p2, a1,b1,p2, w2,a1,n2, f1,b1,u2,w2,n2, n2,b1, a1,n2, n2,f1,l2, a1,f1, l2, n2,l2, a1,l2, t2,a1, t2,f1, t2,a1,y2, t2, t2,n2, i1,a1,v2,p2, i1, t2,v2, g1,b1,p2, n2,g1, a1,f1,b1,n2, t2,a1,n2, f1,i1,l2,a1,g1, f1,l2, a1,l2,n2, i1,v2,g1, a1,l2,p...
Name: not_changes, dtype: object

In [10]:
test = pd.read_csv('raw_data/test_dataset.tsv', sep='\t')
tmp = pd.DataFrame(test['id'].str.split('_', expand=True))
tmp.columns = ['movie', 'scene', 'A', 'movie_id']
for col in ['movie', 'scene', 'movie_id']:
    test[col] = tmp[col].values
test['movie_id'] = test['movie_id'].astype(int)
test['movie'] = test['movie'].map(movie_dict)
test = test.sort_values(by=['movie', 'movie_id'])
test['character'].fillna('无角色', inplace=True)

test['content'] = test['content'].apply(lambda x: x.replace('M国', '美国'))
test['content'] = test['content'].apply(lambda x: x.replace('m国', '美国'))

test.head(10)

,id,content,character,movie,scene,movie_id
0,34170_0002_A_12,穿着背心的b1醒来，看看手机，三点了。,b1,中国机长,0002,12
1,34170_0002_A_14,b1走出卧室。,b1,中国机长,0002,14
2,34170_0003_A_16,b1拿着手机，点开计时功能。,b1,中国机长,0003,16
3,34170_0003_A_17,b1站在淋浴头下面，水从b1的头和脸上冲刷而过。,b1,中国机长,0003,17
4,34170_0003_A_18,b1摈着呼吸。,b1,中国机长,0003,18
5,34170_0003_A_20,b1睁开了眼，喘了口气。,b1,中国机长,0003,20
6,34170_0003_A_21,b1看了看手机，大概四分钟。,b1,中国机长,0003,21
7,34170_0004_A_24,i3躺在被窝里熟睡。,i3,中国机长,0004,24
8,34170_0004_A_25,b1蹲在床边，拉着i3的手，轻声说道：满儿，爸爸今晚下班以后回来给你过生日。,b1,中国机长,0004,25
9,34170_0004_A_26,b1蹲在床边，拉着i3的手，轻声说道：满儿，爸爸今晚下班以后回来给你过生日。,i3,中国机长,0004,26


In [11]:
test['character_new'] = test.apply(lambda row: replace_character(row['character'], row['movie'], datatype='test'), axis=1)
test.head(10)

,id,content,character,movie,scene,movie_id,character_new
0,34170_0002_A_12,穿着背心的b1醒来，看看手机，三点了。,b1,中国机长,0002,12,刘长健
1,34170_0002_A_14,b1走出卧室。,b1,中国机长,0002,14,刘长健
2,34170_0003_A_16,b1拿着手机，点开计时功能。,b1,中国机长,0003,16,刘长健
3,34170_0003_A_17,b1站在淋浴头下面，水从b1的头和脸上冲刷而过。,b1,中国机长,0003,17,刘长健
4,34170_0003_A_18,b1摈着呼吸。,b1,中国机长,0003,18,刘长健
5,34170_0003_A_20,b1睁开了眼，喘了口气。,b1,中国机长,0003,20,刘长健
6,34170_0003_A_21,b1看了看手机，大概四分钟。,b1,中国机长,0003,21,刘长健
7,34170_0004_A_24,i3躺在被窝里熟睡。,i3,中国机长,0004,24,女儿
8,34170_0004_A_25,b1蹲在床边，拉着i3的手，轻声说道：满儿，爸爸今晚下班以后回来给你过生日。,b1,中国机长,0004,25,刘长健
9,34170_0004_A_26,b1蹲在床边，拉着i3的手，轻声说道：满儿，爸爸今晚下班以后回来给你过生日。,i3,中国机长,0004,26,女儿


In [12]:
test['content_new'] = test.apply(lambda row: replace_content(row['content'], row['movie'], datatype='test'), axis=1)
test.head(10)

,id,content,character,movie,scene,movie_id,character_new,content_new
0,34170_0002_A_12,穿着背心的b1醒来，看看手机，三点了。,b1,中国机长,0002,12,刘长健,穿着背心的刘长健醒来，看看手机，三点了。
1,34170_0002_A_14,b1走出卧室。,b1,中国机长,0002,14,刘长健,刘长健走出卧室。
2,34170_0003_A_16,b1拿着手机，点开计时功能。,b1,中国机长,0003,16,刘长健,刘长健拿着手机，点开计时功能。
3,34170_0003_A_17,b1站在淋浴头下面，水从b1的头和脸上冲刷而过。,b1,中国机长,0003,17,刘长健,刘长健站在淋浴头下面，水从刘长健的头和脸上冲刷而过。
4,34170_0003_A_18,b1摈着呼吸。,b1,中国机长,0003,18,刘长健,刘长健摈着呼吸。
5,34170_0003_A_20,b1睁开了眼，喘了口气。,b1,中国机长,0003,20,刘长健,刘长健睁开了眼，喘了口气。
6,34170_0003_A_21,b1看了看手机，大概四分钟。,b1,中国机长,0003,21,刘长健,刘长健看了看手机，大概四分钟。
7,34170_0004_A_24,i3躺在被窝里熟睡。,i3,中国机长,0004,24,女儿,女儿躺在被窝里熟睡。
8,34170_0004_A_25,b1蹲在床边，拉着i3的手，轻声说道：满儿，爸爸今晚下班以后回来给你过生日。,b1,中国机长,0004,25,刘长健,刘长健蹲在床边，拉着女儿的手，轻声说道：满儿，爸爸今晚下班以后回来给你过生日。
9,34170_0004_A_26,b1蹲在床边，拉着i3的手，轻声说道：满儿，爸爸今晚下班以后回来给你过生日。,i3,中国机长,0004,26,女儿,刘长健蹲在床边，拉着女儿的手，轻声说道：满儿，爸爸今晚下班以后回来给你过生日。


In [13]:
test['not_changes'] = test['content_new'].apply(lambda x: check_(x))
test['not_changes'] = test['not_changes'].apply(lambda x: list(set(x)))
test['not_changes_len'] = test['not_changes'].apply(len)
test['not_changes'] = test['not_changes'].apply(lambda x: ','.join(x))

In [14]:
tmp = test[test['not_changes_len'] > 0].copy()
tmp.groupby('movie')['not_changes'].unique()

Series([], Name: not_changes, dtype: float64)

In [16]:
train_df = train[['id', 'movie_id', 'movie', 'scene', 'content_new', 'character_new', 'emotions']].copy()
test_df = test[['id', 'movie_id', 'movie', 'scene', 'content_new', 'character_new']].copy()

train_df = train_df.rename(columns={'content_new': 'content', 'character_new': 'character'})
test_df = test_df.rename(columns={'content_new': 'content', 'character_new': 'character'})

In [17]:
train_df.head()

,id,movie_id,movie,scene,content,character,emotions
14632,32812_0001_A_1,1,一出好戏,0001,一颗卫星的摄像头对准了一望无际的太空深处。,无角色,
14633,32812_0001_A_2,2,一出好戏,0001,一颗巨大的陨石穿过碎石阵朝黑暗深处快速降落。,无角色,
14634,32812_0001_A_3,3,一出好戏,0001,一个美国主持人报道着新闻的画外音：最新消息，美国宇航局之前观测的陨石，受到撞击后改变运行轨迹，正朝地球高速飞来，专家认为，陨石撞击地球的几率非常高。,无角色,
14635,32812_0001_A_4,4,一出好戏,0001,一个B国主持人的画外音：政府已经做好了万全准备，一定尽最大努力确保大家的安全。,无角色,
14636,32812_0001_A_5,5,一出好戏,0001,更多国家主持人不同语言的声音交叉在一切，都在报道陨石事件。,无角色,


In [18]:
test_df.head()

,id,movie_id,movie,scene,content,character
0,34170_0002_A_12,12,中国机长,0002,穿着背心的刘长健醒来，看看手机，三点了。,刘长健
1,34170_0002_A_14,14,中国机长,0002,刘长健走出卧室。,刘长健
2,34170_0003_A_16,16,中国机长,0003,刘长健拿着手机，点开计时功能。,刘长健
3,34170_0003_A_17,17,中国机长,0003,刘长健站在淋浴头下面，水从刘长健的头和脸上冲刷而过。,刘长健
4,34170_0003_A_18,18,中国机长,0003,刘长健摈着呼吸。,刘长健


In [19]:
print(train_df.shape, train.shape)
print(test_df.shape, test.shape)

(42790, 7) (42790, 11)
(21376, 6) (21376, 10)


In [20]:
train_df.to_csv('train_fixed.csv', index=False)
test_df.to_csv('test_fixed.csv', index=False)